<a href="https://colab.research.google.com/github/sathsaraRasantha/Question_Answering_for_Sinhala_Language_using_Deep_NN_Architectures/blob/main/Preprocessing_Pipeline_for_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
import pickle
import re, os, string, typing, gc, json
import spacy
from collections import Counter
nlp = spacy.load('en')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving english_sinhala_combined.csv to english_sinhala_combined.csv


In [ ]:
data=pd.read_csv("english_sinhala_combined.csv")
data=data.drop(['Unnamed: 0',	'context',	'question',	'text'	],axis=1)

In [ ]:
answer_start=[]
answer_end=[]

for i in range(0,20000):
  text=str(data['context_translated'][i])
  answer=str(data['text_translated'][i])
  start=text.find(answer)
  end=start+len(answer)
  answer_start.append(start)
  answer_end.append(end)

In [ ]:
data['answer_start']=answer_start
data['answer_end']=answer_end
data

,context_translated,question_translated,text_translated,answer_start,answer_end
0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
1,"2002 දී ප්‍රදර්ශනය ආරම්භ වූ දා සිට, අයිඩල් ජයග...",ඇමරිකානු අයිඩල් තරඟකරු ක්‍රිස් ඩොට්‍රි කුමන ප්...,උතුරු කැරොලිනාව,-1,14
2,3 වන ශතවර්ෂයේ මැද භාගයේ සිට ජර්මානු ගෝත්‍රිකයන...,7 වන සියවසේ ඉදිකරන ලද දේ,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි,-1,44
3,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි ...,90 දශකයේ දී කැනේ විසින් සාදන ලද බීට්ස් කවුද?,දේශීය කලාකරුවන්,-1,14
4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
...,...,...,...,...,...
19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103
19996,පැරණි ඉංග්‍රීසි භාෂාවෙන් ලතින් භාෂාවෙන් යම් නි...,ලතින් හෝඩිය හඳුන්වාදීමට පෙර පැරණි ඉංග්‍රීසි ලි...,රූනික් පද්ධතිය,-1,13
19997,රූනික් පද්ධතිය... ඇරිස්ටෝෆනේස්ගේ නාට්‍ය ග්‍රීක...,ප්ලේටෝ සහ ඇරිස්ටෝටල් ලියුවේ කුමන ආකාරයේ සාහිත්...,දාර්ශනික පා,-1,10
19998,ආරම්භක හා අවසන් දිනයන් දළ වශයෙන් දකුණු අර්ධගෝල...,දකුණු අර්ධගෝලයේ කුමන කාල පරිච්ඡේදයේදී චිලී ප්‍...,ගිම්හානය,312,320


In [ ]:
df = data[data.answer_start != -1]
df

,context_translated,question_translated,text_translated,answer_start,answer_end
0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
8,ජේන් ඔස්ටන්... ප්‍රධාන තදාසන්න ප්‍රදේශ අතර නිර...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්‍රදේශය කුමක්ද?,මිඩ්ලෝතියන්,56,67
9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්‍ර පාක්ෂිකය...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්‍රාන්...,හත,339,341
21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්‍රංශයේ අවනීතිය ...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,164,178
...,...,...,...,...,...
19990,"1900 වන විට මිනිසුන් 7,531 ක් නගරයේ වාසය කළහ. ...",2006 දී ටක්සන්ගේ ජනගහනය කොපමණද?,"535,000 කි",528,538
19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,320,325
19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,133,151
19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103


In [ ]:
df=df.reset_index()
df

,index,context_translated,question_translated,text_translated,answer_start,answer_end
0,0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
1,4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
2,8,ජේන් ඔස්ටන්... ප්‍රධාන තදාසන්න ප්‍රදේශ අතර නිර...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්‍රදේශය කුමක්ද?,මිඩ්ලෝතියන්,56,67
3,9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්‍ර පාක්ෂිකය...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්‍රාන්...,හත,339,341
4,21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්‍රංශයේ අවනීතිය ...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,164,178
...,...,...,...,...,...,...
8993,19990,"1900 වන විට මිනිසුන් 7,531 ක් නගරයේ වාසය කළහ. ...",2006 දී ටක්සන්ගේ ජනගහනය කොපමණද?,"535,000 කි",528,538
8994,19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,320,325
8995,19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,133,151
8996,19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
df.to_csv('indexed_data.csv')
!cp indexed_data.csv "drive/My Drive/"

In [ ]:
df

,index,context_translated,question_translated,text_translated,answer_start,answer_end
0,0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
1,4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
2,8,ජේන් ඔස්ටන්... ප්‍රධාන තදාසන්න ප්‍රදේශ අතර නිර...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්‍රදේශය කුමක්ද?,මිඩ්ලෝතියන්,56,67
3,9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්‍ර පාක්ෂිකය...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්‍රාන්...,හත,339,341
4,21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්‍රංශයේ අවනීතිය ...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,164,178
...,...,...,...,...,...,...
8993,19990,"1900 වන විට මිනිසුන් 7,531 ක් නගරයේ වාසය කළහ. ...",2006 දී ටක්සන්ගේ ජනගහනය කොපමණද?,"535,000 කි",528,538
8994,19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,320,325
8995,19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,133,151
8996,19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103


In [2]:
from google.colab import files
uploaded = files.upload()

Saving indexed_data.csv to indexed_data.csv


In [13]:

import pandas as pd


In [14]:
df=pd.read_csv("indexed_data.csv")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:

df

,Unnamed: 0,index,context_translated,question_translated,text_translated,answer_start,answer_end
0,0,0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
1,1,4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
2,2,8,ජේන් ඔස්ටන්... ප්‍රධාන තදාසන්න ප්‍රදේශ අතර නිර...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්‍රදේශය කුමක්ද?,මිඩ්ලෝතියන්,56,67
3,3,9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්‍ර පාක්ෂිකය...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්‍රාන්...,හත,339,341
4,4,21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්‍රංශයේ අවනීතිය ...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,164,178
...,...,...,...,...,...,...,...
8993,8993,19990,"1900 වන විට මිනිසුන් 7,531 ක් නගරයේ වාසය කළහ. ...",2006 දී ටක්සන්ගේ ජනගහනය කොපමණද?,"535,000 කි",528,538
8994,8994,19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,320,325
8995,8995,19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,133,151
8996,8996,19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103


In [16]:
def parse_data(dataframe):
  QA_list=[]

  for i in range(0,len(dataframe['index'])):
    QA_dict={}
    QA_dict['id']=i
    QA_dict['context']=dataframe['context_translated'][i]
    QA_dict['question']=dataframe['question_translated'][i]
    QA_dict['label']=[dataframe['answer_start'][0],dataframe['answer_end'][i]]
    QA_list.append(QA_dict)
  return QA_list



In [37]:
import re
from nltk.stem import WordNetLemmatizer
import numpy as np
import matplotlib.pyplot as plt
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer
from nltk import wordnet
from nltk.tokenize import word_tokenize
import string

In [20]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
stemmer = WordNetLemmatizer()

def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\u200d', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
        
        #Replacing special worsds
        #document = re.sub(r'නැකඩ්', " ", document)
        #document = re.sub(r'අඩීචි'," ", document)
        #document = re.sub(r'සොංටැං'," ",document)
        #document = re.sub(r'``', " ", document)
        #document = re.sub(r'[^\w\s]','',document)
        #document = re.sub(r'\_','',document)


        # Converting to Lowercase
        document = document.lower()

        #removing digits from the corpus since FastText unable to embedd digits
        #document= re.sub(r'\b\d+\b', '', document)
        

        # Lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in en_stop]
        tokens = [word for word in tokens if len(word) > 3]

        preprocessed_text = ' '.join(tokens)

        return preprocessed_text

In [38]:
def gather_text_for_vocab(df):
  context=[]
  for row1 in df['context_translated']:
    document1=row1
    text1=preprocess_text(document1)
    context.append(text1)

  
  question=[]
  for row2 in df['question_translated']:
    document2=row2
    text2=preprocess_text(document2)
    question.append(text2)

  table1 = str.maketrans('', '', string.punctuation)
  stripped_context = [w.translate(table1) for w in context]

  table2 = str.maketrans('', '', string.punctuation)
  stripped_question = [w.translate(table2) for w in question]

  lis=stripped_context+stripped_question
  return lis



In [66]:
def build_word_vocab(lis):
   word_vocab=[]
   for sentences in lis:
      tokenized=word_tokenize(sentences)
      word_vocab.append(tokenized)
   
   word2idx = {'__</e>__': 0, '__UNK__': 1} 
   idx2word={}
   for line in lis: 
     words=word_tokenize(line)
     for word in words:
        if word not in word2idx: 
            word2idx[word] = len(word2idx)

   for item in word2idx:
     idx2word[word2idx[item]]=item


   return word2idx, idx2word, word_vocab
   


In [67]:
lis=gather_text_for_vocab(df)
word2idx, idx2word, word_vocab=build_word_vocab(lis)

In [68]:
print(len(word2idx))
print(len(idx2word))
print(len(word_vocab))

59267
59267
17996
